**Name:** Luan Minh Tran

**NetID:** lmt185

**RUID:** 197002611

### Problem Statement
Using PyTorch, set up the pre-trained
network ResNet50. Obtain 10 of your own images that are similar to ImageNet classes
and classify them. Choose 10 images from 5 different classes (2 images per class).
Report the confusion matrix, the accuracy, the f-score, precision and recall of your
classifier. There should be 6 classes representing the 5 classes that your images belong
to as well as an 6th ’other’ class.

### Load required modules

In [1]:
import torch
import os
from skimage import io, transform
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
import itertools

In [2]:
# Load pretrained ResNet50 model
# https://pytorch.org/docs/stable/torchvision/models.html
# Tip: When loading model make sure pretrain argument set to True
# Tip: Good resource for PyTorch projects: https://github.com/pytorch/examples/blob/master/mnist/main.py

device = torch.device('cuda')  # use gpu device
model = models.resnet50(pretrained=True) # load model from torchvision.models
model = model.to(device)  # model operations are sent to GPU
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [3]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
def loadImage(root_dir):
    dataset = []
    for label in os.listdir(root_dir+"/images"):
      
      dataset+=([[os.path.join(root_dir, 'images/'+label, file),label]
                      for file in os.listdir(root_dir + "/images/"+label)
                          if file.endswith('.jpg')])
    return dataset

In [5]:
# Create custom Dataset for your images
# https://pytorch.org/tutorials/beginner/data_loading_tutorial.html

class CustomDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.transform = transform
        self.root_dir = root_dir
        self.dataset = loadImage(root_dir)
        
        # TODO: load image paths and labels located in the root_dir
        # Tip: append image_path and label pairs into self.dataset
    
    def __len__(self):
        return len(self.dataset)
    
    def __getitem__(self, index):
        image_path, label = self.dataset[index]
        
        image = Image.open(image_path)
        
        if self.transform:
            image = self.transform(image)
        
        return image, label

In [6]:
# Load all path data,label
root_dir = '/content/gdrive/My Drive/Computer Vision/Projects/'

transform= transforms.Compose(
    [transforms.Resize(size=(224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


#Load custome dataset 
custom_dataset = CustomDataset(root_dir, transform)
loader = DataLoader(custom_dataset, batch_size=1)

In [7]:
temp = []
for (image, label) in loader:
  temp.append(int(label[0]))

label_t = torch.LongTensor(temp)
label_t = torch.unsqueeze(label_t, dim = 1)
print(label_t)

tensor([[ 71],
        [ 71],
        [ 76],
        [ 76],
        [107],
        [107],
        [340],
        [340],
        [576],
        [576]])


In [8]:
# # Cycle through custom dataset and pass data into the model
# for (image, target) in loader:
#     image = image.to(device)  # enable GPU operations for image
#     prediction = model(image)
#     scores = model(image) #probability for eac class
#     _, preds = torch.max(scores, dim=1)  # please see docs for an explanation
#     print(preds)
#     # TODO: compute metrics on prediction such as accuracy, precision, etc.

In [9]:
epsilons = [0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]

# perturbed_image = image + epsilon∗sign(data_grad) =x + ϵ ∗ sign(∇xJ(θ,x,y))

def fgsm_attack(image, epsilon, data_grad):
    # Collect the element-wise sign of the data gradient
    sign_data_grad = data_grad.sign()
    # Create the perturbed image by adjusting each pixel of the input image
    perturbed_image = image + epsilon*sign_data_grad
    # Adding clipping to maintain [0,1] range
    perturbed_image = torch.clamp(perturbed_image, 0, 1)
    # Return the perturbed image
    return perturbed_image

In [12]:
def check_accuracy(loader, model, epsilon):
    num_correct = 0
    num_samples = 0
    adv_examples = []
    
    # with torch.no_grad():
    for (data, _), label in zip(loader, label_t):
        data = data.to(device=device)
        label = label.to(device=device)
        
        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        scores = model(data)
        init_pred = scores.max(1, keepdim=True)[1]
        # print("Hello init: ", init_pred)
        if init_pred.item() != label.item(): # if the intial prediction is wrong => no need to attack
          continue

        loss = F.cross_entropy(scores, label)

        # Zero all existing gradients
        model.zero_grad()

        loss.backward() # Calculate gradients of model in backward pass

        data_grad = data.grad.data # Collect datagrad

        perturbed_data = fgsm_attack(data, epsilon, data_grad) # Perform attack

        scores = model(perturbed_data) # Re-model with perturbed_data 

        final_pred = scores.max(1, keepdim=True)[1]

        if final_pred.item() == label.item():
          num_correct += 1

          # In case epsilon == 0
          if (len(adv_examples) < 10):
            adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
            adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        else:
          # Save for visualization later:
          if len(adv_examples) < 10:
            adv_ex = perturbed_data.squeeze().detach().cpu().numpy()
            adv_examples.append( (init_pred.item(), final_pred.item(), adv_ex) )
        # print("Hello final: ", final_pred)
        # print(label)
        # num_correct += (predictions == label).sum()
        # num_samples += predictions.size(0)
    final_acc = num_correct/float(len(loader))
    print("Epsilon: {}\tTest Accuracy = {} / {} = {}".format(epsilon, num_correct, len(loader), final_acc))

    return final_acc, adv_examples

In [13]:
accuracies = []
examples = []

# Run test for each epsilon
for eps in epsilons:
    acc, ex = check_accuracy(loader, model, eps)
    accuracies.append(acc)
    examples.append(ex)

Epsilon: 0	Test Accuracy = 6 / 10 = 0.6
Epsilon: 0.05	Test Accuracy = 4 / 10 = 0.4
Epsilon: 0.1	Test Accuracy = 4 / 10 = 0.4
Epsilon: 0.15	Test Accuracy = 4 / 10 = 0.4
Epsilon: 0.2	Test Accuracy = 3 / 10 = 0.3
Epsilon: 0.25	Test Accuracy = 3 / 10 = 0.3
Epsilon: 0.3	Test Accuracy = 2 / 10 = 0.2


In [14]:
cnt = 0
plt.figure(figsize=(20,20))
for i in range(len(epsilons)):
    for j in range(len(examples[i])):
        cnt += 1
        plt.subplot(len(epsilons),len(examples[0]),cnt)
        plt.xticks([], [])
        plt.yticks([], [])
        if j == 0:
            plt.ylabel("Eps: {}".format(epsilons[i]), fontsize=14)
        orig,adv,ex = examples[i][j]
        plt.title("{} -> {}".format(orig, adv), fontsize = 14)
        plt.imshow(np.transpose(ex, (1,2,0)))
plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.